# Crashspot – Week 4 Starter
**Focus:** Kernel Density Estimation (KDE) heatmaps for Monroe and Louisiana

- Monroe KDE (PNG + interactive HTML)
- Louisiana KDE (PNG + interactive HTML)
- Side‑by‑side PNG comparison
- GeoTIFF rasters for QGIS/ArcGIS



In [11]:
# 0) Paths & folders
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if (Path.cwd().name.startswith('Crashspot_')) else Path.cwd()

DATA_CLEAN = PROJECT_ROOT / "data_clean"
DOCS = PROJECT_ROOT / "docs"
DOCS_MAPS = DOCS / "maps"
FIGS = PROJECT_ROOT / "outputs" / "figures"
for p in [DOCS_MAPS, FIGS, DATA_CLEAN]:
    p.mkdir(parents=True, exist_ok=True)

LA_FILE = DATA_CLEAN / "fars_la_2022_2023.geojson"
MO_FILE = DATA_CLEAN / "fars_monroe_2022_2023.geojson"

PNG_MONROE = FIGS / "monroe_kde.png"
PNG_LA     = FIGS / "la_kde.png"
PNG_SIDE   = FIGS / "kde_monroe_vs_louisiana.png"
HTML_MONROE = DOCS_MAPS / "week4_monroe_heatmap.html"
HTML_LA     = DOCS_MAPS / "week4_louisiana_heatmap.html"
TIF_MONROE  = DATA_CLEAN / "monroe_kde.tif"
TIF_LA      = DATA_CLEAN / "la_kde.tif"

print("PROJECT_ROOT:", PROJECT_ROOT.resolve())
print("Exists:", LA_FILE.exists(), MO_FILE.exists())

PROJECT_ROOT: /Users/himalranabhat/Desktop/Crashspot
Exists: True True


In [12]:
# 1) Imports
import numpy as np, pandas as pd, geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point
print("Libraries ready")

Libraries ready


## Load Monroe & Louisiana crashes (cleaned in Week 1)

In [13]:
# 2) Load (and ensure CRS)
if not LA_FILE.exists() or not MO_FILE.exists():
    raise FileNotFoundError("Missing cleaned files. Ensure Week 1 outputs exist in data_clean/.")
la = gpd.read_file(LA_FILE).dropna(subset=["geometry"]).to_crs(4326)
mo = gpd.read_file(MO_FILE).dropna(subset=["geometry"]).to_crs(4326)
print("Counts -> LA:", len(la), "| Monroe:", len(mo))

Counts -> LA: 1607 | Monroe: 60


## Project to UTM 15N (EPSG:32615) for meter-based KDE

In [21]:
# 3) Project to meters
def clean_project(gdf_wgs84, epsg=32615):
    # keep valid geometries
    g = gdf_wgs84[gdf_wgs84.geometry.notna()].copy()
    g = g[g.geometry.is_valid].to_crs(epsg)
    # drop any non-finite projected coords
    x = g.geometry.x.to_numpy()
    y = g.geometry.y.to_numpy()
    m = np.isfinite(x) & np.isfinite(y)
    return g.iloc[m]

la_m = clean_project(la, 32615)
mo_m = clean_project(mo, 32615)

print("CRS (LA):", la_m.crs, "| CRS (Monroe):", mo_m.crs)
print("Counts after cleaning → LA:", len(la_m), "| Monroe:", len(mo_m))

CRS (LA): EPSG:32615 | CRS (Monroe): EPSG:32615
Counts after cleaning → LA: 1603 | Monroe: 60


## Bandwidth selection (GridSearchCV with fallback)

In [22]:
# 4) Robust bandwidth selection (with safeguards + fallback)
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity

def _finite_xy(gdf_m):
    xy = np.column_stack([gdf_m.geometry.x.values, gdf_m.geometry.y.values]).astype(float)
    m = np.isfinite(xy).all(axis=1)
    return xy[m]

def pick_bandwidth(gdf_m, candidates=(200, 400, 600, 800, 1000, 1500), fallback=600.0):
    X = _finite_xy(gdf_m)
    n = len(X)

    # Too few points? use fallback
    if n < 10:
        print(f"[bw] only {n} points → fallback {fallback} m")
        return float(fallback)

    # Set a safe CV (at least 2, at most 5, and not larger than n)
    cv = max(2, min(5, n // 10 or 2))
    try:
        grid = GridSearchCV(
            KernelDensity(kernel="gaussian"),
            {"bandwidth": np.array(candidates, dtype=float)},
            cv=cv,
            error_score="raise"
        )
        grid.fit(X)
        bw = float(grid.best_params_["bandwidth"])
        print(f"[bw] n={n}, cv={cv} → {bw} m")
        return bw
    except Exception as e:
        print(f"[bw] grid search failed ({e}) → fallback {fallback} m")
        return float(fallback)

bw_mo = pick_bandwidth(mo_m)
bw_la = pick_bandwidth(la_m)
print("Chosen bandwidths (m): Monroe =", int(bw_mo), "| Louisiana =", int(bw_la))


[bw] n=60, cv=5 → 1500.0 m
[bw] n=1603, cv=5 → 1500.0 m
Chosen bandwidths (m): Monroe = 1500 | Louisiana = 1500


## KDE rasterization to grid (returns X, Y, Z)

In [25]:
# 5) KDE raster (memory/finites safe)
def kde_raster_safe(gdf_m, bandwidth, cell=500, expand=2000, max_cells=1_500_000):
    """
    Build a KDE raster from a GeoDataFrame in a projected CRS (meters).
    - Uses finite XY only to compute bounds
    - Auto-increases cell size until nx*ny <= max_cells
    """
    # finite XY from geometries
    xy = np.column_stack([gdf_m.geometry.x.values, gdf_m.geometry.y.values]).astype(float)
    mask = np.isfinite(xy).all(axis=1)
    xy = xy[mask]
    if xy.size == 0:
        raise ValueError("No finite coordinates available after cleaning.")

    # bounds from finite XY
    xmin, ymin = xy.min(axis=0)
    xmax, ymax = xy.max(axis=0)
    xmin -= float(expand); ymin -= float(expand)
    xmax += float(expand); ymax += float(expand)

    # guard against degenerate extents
    width  = max(1.0, xmax - xmin)
    height = max(1.0, ymax - ymin)

    # initial grid size
    cell = float(cell)
    nx = max(1, int(np.ceil(width  / cell)))
    ny = max(1, int(np.ceil(height / cell)))

    # auto-grow until within limit
    while nx * ny > max_cells:
        cell *= 1.5
        nx = max(1, int(np.ceil(width  / cell)))
        ny = max(1, int(np.ceil(height / cell)))

    xs = np.linspace(xmin, xmax, nx)
    ys = np.linspace(ymin, ymax, ny)
    xx, yy = np.meshgrid(xs, ys)
    grid = np.column_stack([xx.ravel(), yy.ravel()])

    kde = KernelDensity(bandwidth=float(bandwidth), kernel="gaussian").fit(xy)
    Z = np.exp(kde.score_samples(grid)).reshape(ny, nx)
    return xs, ys, Z, cell

# Monroe: fine resolution
xs_mo, ys_mo, Z_mo, cell_mo = kde_raster_safe(mo_m, bw_mo, cell=120,  expand=2000, max_cells=1_200_000)

# Louisiana: start coarse; function will upsize cell if needed
xs_la, ys_la, Z_la, cell_la = kde_raster_safe(la_m, bw_la, cell=2000, expand=4000, max_cells=1_200_000)

print("Monroe Z shape:", Z_mo.shape, "cell(m):", int(cell_mo))
print("LA     Z shape:", Z_la.shape, "cell(m):",    int(cell_la))



Monroe Z shape: (402, 365) cell(m): 120
LA     Z shape: (213, 229) cell(m): 2000


## Save static PNGs (Monroe, LA, side-by-side)

In [27]:
# 6) Static plots
def plot_kde(xs, ys, Z, title, outpng):
    plt.figure(figsize=(7,6))
    plt.imshow(Z, extent=[xs.min(), xs.max(), ys.min(), ys.max()],
    origin="lower", cmap="hot")
    plt.title(title); plt.axis("off"); plt.tight_layout()
    plt.savefig(outpng, dpi=200); plt.close()

plot_kde(xs_mo, ys_mo, Z_mo, f"Monroe KDE (bw={int(bw_mo)} m)", PNG_MONROE)
plot_kde(xs_la, ys_la, Z_la, f"Louisiana KDE (bw={int(bw_la)} m)", PNG_LA)

plt.figure(figsize=(12,5))
for i,(xs,ys,Z,title) in enumerate([(xs_mo,ys_mo,Z_mo,"Monroe"), (xs_la,ys_la,Z_la,"Louisiana")]):
    ax = plt.subplot(1,2,i+1)
    ax.imshow(Z, extent=[xs.min(), xs.max(), ys.min(), ys.max()], origin="lower", cmap="hot")
    ax.set_title(title); ax.axis("off")
plt.tight_layout()
plt.savefig(PNG_SIDE, dpi=220); plt.close()

print("Saved:", PNG_MONROE, "\nSaved:", PNG_LA, "\nSaved:", PNG_SIDE)

Saved: /Users/himalranabhat/Desktop/Crashspot/outputs/figures/monroe_kde.png 
Saved: /Users/himalranabhat/Desktop/Crashspot/outputs/figures/la_kde.png 
Saved: /Users/himalranabhat/Desktop/Crashspot/outputs/figures/kde_monroe_vs_louisiana.png


## Interactive Folium heatmaps (point-based)

In [ ]:
# 7) Interactive Folium heatmaps
def folium_heatmap(gdf_wgs84, out_html, zoom=12):
    m = folium.Map(location=[gdf_wgs84.geometry.y.mean(), gdf_wgs84.geometry.x.mean()],
    zoom_start=zoom, tiles="CartoDB Positron")
    pts = [[pt.y, pt.x] for pt in gdf_wgs84.geometry]
    HeatMap(pts, radius=18, blur=22, min_opacity=0.15).add_to(m)
    m.save(out_html); 
    print("Saved:", out_html)
    return m

m1 = folium_heatmap(mo.to_crs(4326), HTML_MONROE, zoom=12)
m2 = folium_heatmap(la.to_crs(4326), HTML_LA, zoom=6)
m1

Saved: /Users/himalranabhat/Desktop/Crashspot/docs/maps/week4_monroe_heatmap.html
Saved: /Users/himalranabhat/Desktop/Crashspot/docs/maps/week4_louisiana_heatmap.html


## (Optional) Export KDE GeoTIFFs for QGIS/ArcGIS

In [5]:
# 8) GeoTIFF export
import rasterio
from rasterio.transform import from_origin

def export_geotiff(xs, ys, Z, out_path, crs="EPSG:32615"):
    """
    Export a KDE raster to GeoTIFF.
    
    xs, ys = coordinate arrays (from np.linspace in kde_raster)
    Z = raster intensity array
    out_path = output .tif file
    crs = projection (default: UTM zone 15N for Monroe/Louisiana)
    """
    # Ensure float32 for raster storage
    Z = Z.astype("float32")
    
    # Resolution (cell size in meters)
    xres = (xs[-1] - xs[0]) / (len(xs) - 1)
    yres = (ys[-1] - ys[0]) / (len(ys) - 1)

    # Affine transform: top-left corner (origin)
    transform = from_origin(xs[0], ys[-1], xres, yres)

    with rasterio.open(
        out_path, "w",
        driver="GTiff",
        height=Z.shape[0],
        width=Z.shape[1],
        count=1,
        dtype="float32",
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(Z, 1)

    print(f"✅ Exported GeoTIFF: {out_path}")
